In [1]:
# loading EASYMORE
from easymore.easymore import easymore

# initializing EASYMORE object
esmr = easymore()

# specifying EASYMORE objects
# name of the case; the temporary, remapping and remapped file names include case name
esmr.case_name                = 'North_Asia'              
# temporary path that the EASYMORE generated GIS files and remapped file will be saved
esmr.temp_dir                 = '../temporary/'
# name of target shapefile that the source netcdf files should be remapped to
esmr.target_shp               = '/Users/shg096/Desktop/MERIT_Hydro/cat_final/cat_pfaf_35_MERIT_Hydro_v07_Basins_v01_bugfix1_fixed.shp'
# name of netCDF file(s); multiple files can be specified with *
esmr.source_nc                = '../data/Source_nc_ERA5/ERA5_subset_NorthAsia_*.nc'
# name of variables from source netCDF file(s) to be remapped
esmr.var_names                = ['airtemp']
# rename the variables from source netCDF file(s) in the remapped files;
# it will be the same as source if not provided
esmr.var_names_remapped       = ['temperature']
# name of variable longitude in source netCDF files
esmr.var_lon                  = 'longitude'
# name of variable latitude in source netCDF files
esmr.var_lat                  = 'latitude'
# name of variable time in source netCDF file; should be always time
esmr.var_time                 = 'time'
# location where the remapped netCDF file will be saved
esmr.output_dir               = '../output/'
# format of the variables to be saved in remapped files,
# if one format provided it will be expanded to other variables
esmr.format_list              = ['f4']
# fill values of the variables to be saved in remapped files,
# if one value provided it will be expanded to other variables
esmr.fill_value_list          = ['-9999.00']
# if required that the remapped values to be saved as csv as well
esmr.save_csv                 = True
# if uncommented EASYMORE will use this and skip GIS tasks
# esmr.remap_csv                = '../temporary/ERA5_Medicine_Hat_remapping.csv'

# execute EASYMORE
esmr.nc_remapper()



no author name is provide and the author name is changed to (author name)!
EASYMORE will remap variable  airtemp  from source file to variable  temperature  in remapped NeCDF file
EASYMORE detects that target shapefile is in WGS84 (epsg:4326)
EASYMORE detects that no field for ID is provided in sink/target shapefile
arbitarary values of ID are added in the field ID_t
EASYMORE detects that either of the fields for latitude or longitude is not provided in sink/target shapefile
calculating centroid of shapes in equal area projection


KeyboardInterrupt: 

In [ ]:
# visualize the result
import geopandas as gpd
import xarray as xr
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
font = {'family' : 'Times New Roman',
         'weight' : 'bold',
         'size'   : 24}
matplotlib.rc('font', **font)

# visualize the original netCDF for the first file, first time step
shp_target = gpd.read_file(esmr.target_shp) # load the target shapefile
# load the first source netcdf file
nc_name = sorted(glob.glob(esmr.source_nc))
ds_source = xr.open_dataset(nc_name[0])
# subset the region of interest:
lat_lim = np.array([ 48.2 , 51.7 ]) # limit for lat
lon_lim = np.array([ -116.5 , -110.5 ]) # limit for lon
lats = np.array(ds_source.variables[esmr.var_lat][:] )
lons = np.array(ds_source.variables[esmr.var_lon][:])
lat_low = np.argmin( np.abs( lats - lat_lim[0] ) )
lat_up = np.argmin( np.abs( lats - lat_lim[1] ) )
lon_low = np.argmin( np.abs( lons - lon_lim[0] ) )
lon_up = np.argmin( np.abs( lons - lon_lim[1] ) )
if lat_up < lat_low:
    temp = lat_low
    lat_low = lat_up
    lat_up = temp
if lon_up < lon_low:
    temp = lon_low
    lon_low = lon_up
    lon_up = temp
ds_source = ds_source.isel(latitude=np.arange(lat_low,lat_up))
ds_source = ds_source.isel(longitude=np.arange(lon_low,lon_up))
# plotting
f, axes = plt.subplots(1,1,figsize=(15,15))
ds_source.airtemp[0,:,:].plot( ax = axes)
shp_target.geometry.boundary.plot(color=None,edgecolor='k',linewidth = 1, ax = axes)
plt.savefig('../fig/Example1_A.png')

# visualize the remapped netCDF for the first file, first time step
# target nc file
nc_names = sorted(glob.glob (esmr.output_dir + esmr.case_name + '*.nc'))
ds       = xr.open_dataset(nc_names[0]) # the first netcdf file
values   = ds.temperature [0,:] # the first time frame of the first 
IDs      = ds.ID [:] # get the ID
# create a data frame for the model simulation
df = pd.DataFrame()
df ['value'] = values
df ['ID_t']    = IDs  # .astype(int)
df = df.sort_values (by = 'ID_t')
# load the shape file target that is generated by EASYMORE (with consistent IDs)
shp_target = gpd.read_file(esmr.temp_dir+ esmr.case_name + '_target_shapefile.shp') # load the target shapefile
shp_target ['ID_t'] = shp_target ['ID_t'].astype(float)
shp_target = shp_target.sort_values(by='ID_t')# sort on values
shp_target = pd.merge_asof(shp_target, df, on='ID_t', direction='nearest')
shp_target = shp_target.set_geometry('geometry') #bring back the geometry filed; pd to gpd
# plotting
f, axes = plt.subplots(1,1,figsize=(15,15))
ds_source.airtemp[0,:,:].plot( ax = axes)
ds_source.airtemp[0,:,:].plot( ax = axes, alpha = 1, add_colorbar=False)
shp_target.plot(column= 'value', edgecolor='k',linewidth = 1, ax = axes )#, legend=True)
plt.savefig('../fig/Example1_B.png')



In [1]:
# visualize the result
import geopandas as gpd
import xarray as xr
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
font = {'family' : 'Times New Roman',
         'weight' : 'bold',
         'size'   : 24}
matplotlib.rc('font', **font)

ds = xr.open_dataset('/Users/shg096/Desktop/EASYMORE/data/Source_nc_ERA5/ERA5_subset_NorthAsia_19800101.nc')

print(ds)


<xarray.Dataset>
Dimensions:    (latitude: 146, longitude: 528, time: 24)
Coordinates:
  * longitude  (longitude) float32 58.75 59.0 59.25 59.5 ... 190.0 190.2 190.5
  * latitude   (latitude) float32 81.5 81.25 81.0 80.75 ... 45.75 45.5 45.25
  * time       (time) datetime64[ns] 1980-01-01 ... 1980-01-01T23:00:00
Data variables:
    LWRadAtm   (time, latitude, longitude) float32 ...
    SWRadAtm   (time, latitude, longitude) float32 ...
    pptrate    (time, latitude, longitude) float32 ...
    airpres    (time, latitude, longitude) float32 ...
    airtemp    (time, latitude, longitude) float32 ...
    spechum    (time, latitude, longitude) float32 ...
    windspd    (time, latitude, longitude) float32 ...
Attributes:
    History:                                  Created Mon Jul 19 00:07:20 2021
    Language:                                 Written using Python
    Reason:                                   (1) ERA5 surface and pressure f...
    Conventions (pressure level (10m) data): 

In [4]:
# loading EASYMORE
from easymore.easymore import easymore

# initializing EASYMORE object
esmr = easymore()

print(esmr.__version__)

AttributeError: 'easymore' object has no attribute '__version__'

In [2]:
print(esmr.)

AttributeError: 'easymore' object has no attribute 'version'